### Load libraries

In [80]:
#load libraries
import pandas as pd
import numpy as np
import matplotlib as plt
import os
import seaborn as sns
import geopandas as gpd

In [81]:
#read the ICER data
icer = pd.read_excel("../data/raw/ICER_data.xlsx")

In [82]:
#read in the data for life expectancy
le_district = pd.read_csv("../data/processed/le_combined.csv")
le_nat = pd.read_csv("../data/processed/le_calc_national.csv")
df_map = gpd.read_file("../data/processed/final_combined_divisions/final_combined_divisions.shp")

In [83]:
#rename first column as treatment
icer = icer.rename(columns={"Unnamed: 0": "Treatment"})

In [84]:
icer.head()

,Treatment,burden,qaly_lost,healthcare_cost,payer_cost
0,non_vaccine,382416,184088.54,26618814,16066984
1,vaccine,264217,127189.68,352828922,345497041


### Calculate district-level ICER based on the available ICER (average) from 2020-2024 in Indonesia (from research paper), nationwide, adjusted by life expectancy to obtain district-level ICER, assuming intervention costs remain approximately the same (similar) --> life expectancy is to adjust based on QALYs. All prices are adjusted to 2018 USD based on PPP and discounted by 3%. Ratio of dengue cases per district per year to nationwide dengue cases total from 2020-2024 is used to adjust QALYs and total intervention cost per district

In [85]:
le_district.head()

,District,Year,Life Expectancy
0,ACEH,2022,70.18
1,SIMEULUE,2022,65.48
2,ACEH SINGKIL,2022,67.65
3,ACEH SELATAN,2022,64.64
4,ACEH TENGGARA,2022,68.48


In [86]:
#rename as district_life_exp
le_district = le_district.rename(columns={"Life Expectancy": "district_le"})

In [87]:
le_nat.head()

,Year,Life Expectancy
0,2019,69.213148
1,2020,69.363809
2,2021,69.474243
3,2022,69.747513
4,2023,70.018191


In [88]:
#rename as nat_le
le_nat = le_nat.rename(columns={"Life Expectancy": "nat_le"})

In [89]:
#merge le_district and le_nat by year
le_final = pd.merge(le_district, le_nat, on = "Year")

#replace all the NA values

#make sure all the life expectancy values are all floats
le_final["district_le"] = le_final["district_le"].astype(float)
le_final["nat_le"] = le_final["nat_le"].astype(float)

#calculate the le_factor
le_final["le_factor"] = le_final["district_le"] / le_final["nat_le"]

In [90]:
#create another column in df_map to calculate the discounting factor for each district in dki jakarta and west java
df = pd.merge(df_map, le_final, on = ("District", "Year"))

In [91]:
#remove the district_le and nat_le columns
df_final = df.drop(columns = ["district_le", "nat_le"])

In [92]:
df_final["healthcare_cost_non_vaccine"] = icer["healthcare_cost"].iloc[0]
df_final["healthcare_cost_vaccine"] = icer["healthcare_cost"].iloc[1]
df_final["payer_cost_non_vaccine"] = icer["payer_cost"].iloc[0]
df_final["payer_cost_vaccine"] = icer["payer_cost"].iloc[1]
df_final["qaly_lost_non_vaccine"] = icer["qaly_lost"].iloc[0]
df_final["qaly_lost_vaccine"] = icer["qaly_lost"].iloc[1]
df_final["burden_non_vaccine"] = icer["burden"].iloc[0]
df_final["burden_vaccine"] = icer["burden"].iloc[1]

In [93]:
#calculate ratio of district cases to national cases
df_final["cases_ratio"] = df_final["Cases"] / (df_final["burden_non_vaccine"] + df_final["burden_vaccine"])

In [94]:
#calculate district-level adjusted ICER for payers and healthcare
# Scale QALYs lost using le_factor
df_final['qaly_lost_scaled_vaccine'] = df_final['qaly_lost_vaccine'] * df_final['le_factor'] * df_final["cases_ratio"]
df_final['qaly_lost_scaled_nonvaccine'] = df_final['qaly_lost_non_vaccine'] * df_final['le_factor'] * df_final["cases_ratio"]

In [95]:
# Scale costs
df_final['healthcare_cost_vaccine_scaled'] = df_final['healthcare_cost_vaccine'] * df_final['cases_ratio']
df_final['healthcare_cost_nonvaccine_scaled'] = df_final['healthcare_cost_non_vaccine'] * df_final['cases_ratio']
df_final['payer_cost_vaccine_scaled'] = df_final['payer_cost_vaccine'] * df_final['cases_ratio']
df_final['payer_cost_nonvaccine_scaled'] = df_final['payer_cost_non_vaccine'] * df_final['cases_ratio']

In [96]:
#calculate the ICER aft adjusting for qalys
df_final['ICER_healthcare_adjusted'] = abs((df_final["healthcare_cost_non_vaccine"] - df_final["healthcare_cost_vaccine"]) / (df_final["qaly_lost_scaled_nonvaccine"] - df_final["qaly_lost_scaled_vaccine"]))
df_final['ICER_payer_adjusted'] = abs((df_final["payer_cost_non_vaccine"] - df_final["payer_cost_vaccine"]) / (df_final["qaly_lost_scaled_nonvaccine"] - df_final["qaly_lost_scaled_vaccine"]))

In [97]:
df_final.head()

,District,x,y,Year,Area_sq_km,HDI,Cases,Population,Pop_den,geometry,...,burden_vaccine,cases_ratio,qaly_lost_scaled_vaccine,qaly_lost_scaled_nonvaccine,healthcare_cost_vaccine_scaled,healthcare_cost_nonvaccine_scaled,payer_cost_vaccine_scaled,payer_cost_nonvaccine_scaled,ICER_healthcare_adjusted,ICER_payer_adjusted
0,BANDUNG,107.610841,-7.099969,2020,1767.96,72.39,9180,14495160,8198.805403,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,264217,0.014197,1914.116318,2770.404630,5.008977e+06,377897.064517,4.904889e+06,228096.792338,380958.263056,384718.619183
1,BANDUNG,107.610841,-7.099969,2021,1767.96,72.73,8008,14662620,8293.524740,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,264217,0.012384,1671.396833,2419.103521,4.369486e+06,329651.382642,4.278687e+06,198975.938240,436280.848042,440587.281368
2,BANDUNG,107.610841,-7.099969,2022,1767.96,73.16,16764,14830092,8388.250865,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,264217,0.025925,3498.914578,5064.169328,9.147111e+06,690094.377949,8.957032e+06,416537.541041,208407.039215,210464.179754
3,BANDUNG,107.610841,-7.099969,2023,1767.96,73.74,4020,14997564,8482.976990,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,264217,0.006217,838.730815,1213.940716,2.193473e+06,165484.335442,2.147892e+06,99885.523442,869406.983418,877988.710588
4,BANDUNG BARAT,107.414953,-6.897056,2020,1305.77,68.08,3864,7153344,5478.257273,"POLYGON ((107.40945 -6.68851, 107.40986 -6.688...",...,264217,0.005976,792.641306,1147.232864,2.108354e+06,159062.555261,2.064541e+06,96009.368801,919960.166026,929040.892662


In [98]:
# Scale total cost to produce district-level cost per year based on cases per district per year / total num of cases over 2020-2024 nationwide
df_final['cost_factor'] = df_final['Cases'] / (df_final['burden_non_vaccine'] + df_final['burden_vaccine'])


In [99]:
df_final.head()

,District,x,y,Year,Area_sq_km,HDI,Cases,Population,Pop_den,geometry,...,cases_ratio,qaly_lost_scaled_vaccine,qaly_lost_scaled_nonvaccine,healthcare_cost_vaccine_scaled,healthcare_cost_nonvaccine_scaled,payer_cost_vaccine_scaled,payer_cost_nonvaccine_scaled,ICER_healthcare_adjusted,ICER_payer_adjusted,cost_factor
0,BANDUNG,107.610841,-7.099969,2020,1767.96,72.39,9180,14495160,8198.805403,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,0.014197,1914.116318,2770.404630,5.008977e+06,377897.064517,4.904889e+06,228096.792338,380958.263056,384718.619183,0.014197
1,BANDUNG,107.610841,-7.099969,2021,1767.96,72.73,8008,14662620,8293.524740,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,0.012384,1671.396833,2419.103521,4.369486e+06,329651.382642,4.278687e+06,198975.938240,436280.848042,440587.281368,0.012384
2,BANDUNG,107.610841,-7.099969,2022,1767.96,73.16,16764,14830092,8388.250865,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,0.025925,3498.914578,5064.169328,9.147111e+06,690094.377949,8.957032e+06,416537.541041,208407.039215,210464.179754,0.025925
3,BANDUNG,107.610841,-7.099969,2023,1767.96,73.74,4020,14997564,8482.976990,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,0.006217,838.730815,1213.940716,2.193473e+06,165484.335442,2.147892e+06,99885.523442,869406.983418,877988.710588,0.006217
4,BANDUNG BARAT,107.414953,-6.897056,2020,1305.77,68.08,3864,7153344,5478.257273,"POLYGON ((107.40945 -6.68851, 107.40986 -6.688...",...,0.005976,792.641306,1147.232864,2.108354e+06,159062.555261,2.064541e+06,96009.368801,919960.166026,929040.892662,0.005976


In [100]:
# Scale costs
df_final['healthcare_cost_vaccine_scaled'] = df_final['healthcare_cost_vaccine'] * df_final['cost_factor']
df_final['healthcare_cost_nonvaccine_scaled'] = df_final['healthcare_cost_non_vaccine'] * df_final['cost_factor']
df_final['payer_cost_vaccine_scaled'] = df_final['payer_cost_vaccine'] * df_final['cost_factor']
df_final['payer_cost_nonvaccine_scaled'] = df_final['payer_cost_non_vaccine'] * df_final['cost_factor']

df_final.head()

,District,x,y,Year,Area_sq_km,HDI,Cases,Population,Pop_den,geometry,...,cases_ratio,qaly_lost_scaled_vaccine,qaly_lost_scaled_nonvaccine,healthcare_cost_vaccine_scaled,healthcare_cost_nonvaccine_scaled,payer_cost_vaccine_scaled,payer_cost_nonvaccine_scaled,ICER_healthcare_adjusted,ICER_payer_adjusted,cost_factor
0,BANDUNG,107.610841,-7.099969,2020,1767.96,72.39,9180,14495160,8198.805403,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,0.014197,1914.116318,2770.404630,5.008977e+06,377897.064517,4.904889e+06,228096.792338,380958.263056,384718.619183,0.014197
1,BANDUNG,107.610841,-7.099969,2021,1767.96,72.73,8008,14662620,8293.524740,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,0.012384,1671.396833,2419.103521,4.369486e+06,329651.382642,4.278687e+06,198975.938240,436280.848042,440587.281368,0.012384
2,BANDUNG,107.610841,-7.099969,2022,1767.96,73.16,16764,14830092,8388.250865,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,0.025925,3498.914578,5064.169328,9.147111e+06,690094.377949,8.957032e+06,416537.541041,208407.039215,210464.179754,0.025925
3,BANDUNG,107.610841,-7.099969,2023,1767.96,73.74,4020,14997564,8482.976990,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,0.006217,838.730815,1213.940716,2.193473e+06,165484.335442,2.147892e+06,99885.523442,869406.983418,877988.710588,0.006217
4,BANDUNG BARAT,107.414953,-6.897056,2020,1305.77,68.08,3864,7153344,5478.257273,"POLYGON ((107.40945 -6.68851, 107.40986 -6.688...",...,0.005976,792.641306,1147.232864,2.108354e+06,159062.555261,2.064541e+06,96009.368801,919960.166026,929040.892662,0.005976


In [101]:
# Compute incremental costs
df_final['incremental_healthcare_cost'] = df_final['healthcare_cost_vaccine_scaled'] - df_final['healthcare_cost_nonvaccine_scaled']
df_final['incremental_payer_cost'] = df_final['payer_cost_vaccine_scaled'] - df_final['payer_cost_nonvaccine_scaled']

df_final.head()

,District,x,y,Year,Area_sq_km,HDI,Cases,Population,Pop_den,geometry,...,qaly_lost_scaled_nonvaccine,healthcare_cost_vaccine_scaled,healthcare_cost_nonvaccine_scaled,payer_cost_vaccine_scaled,payer_cost_nonvaccine_scaled,ICER_healthcare_adjusted,ICER_payer_adjusted,cost_factor,incremental_healthcare_cost,incremental_payer_cost
0,BANDUNG,107.610841,-7.099969,2020,1767.96,72.39,9180,14495160,8198.805403,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,2770.404630,5.008977e+06,377897.064517,4.904889e+06,228096.792338,380958.263056,384718.619183,0.014197,4.631079e+06,4.676792e+06
1,BANDUNG,107.610841,-7.099969,2021,1767.96,72.73,8008,14662620,8293.524740,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,2419.103521,4.369486e+06,329651.382642,4.278687e+06,198975.938240,436280.848042,440587.281368,0.012384,4.039835e+06,4.079711e+06
2,BANDUNG,107.610841,-7.099969,2022,1767.96,73.16,16764,14830092,8388.250865,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,5064.169328,9.147111e+06,690094.377949,8.957032e+06,416537.541041,208407.039215,210464.179754,0.025925,8.457017e+06,8.540494e+06
3,BANDUNG,107.610841,-7.099969,2023,1767.96,73.74,4020,14997564,8482.976990,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,1213.940716,2.193473e+06,165484.335442,2.147892e+06,99885.523442,869406.983418,877988.710588,0.006217,2.027989e+06,2.048007e+06
4,BANDUNG BARAT,107.414953,-6.897056,2020,1305.77,68.08,3864,7153344,5478.257273,"POLYGON ((107.40945 -6.68851, 107.40986 -6.688...",...,1147.232864,2.108354e+06,159062.555261,2.064541e+06,96009.368801,919960.166026,929040.892662,0.005976,1.949291e+06,1.968532e+06


In [102]:
# Compute incremental QALY
df_final['incremental_qaly'] = df_final['qaly_lost_scaled_nonvaccine'] - df_final['qaly_lost_scaled_vaccine']

df_final.head()

,District,x,y,Year,Area_sq_km,HDI,Cases,Population,Pop_den,geometry,...,healthcare_cost_vaccine_scaled,healthcare_cost_nonvaccine_scaled,payer_cost_vaccine_scaled,payer_cost_nonvaccine_scaled,ICER_healthcare_adjusted,ICER_payer_adjusted,cost_factor,incremental_healthcare_cost,incremental_payer_cost,incremental_qaly
0,BANDUNG,107.610841,-7.099969,2020,1767.96,72.39,9180,14495160,8198.805403,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,5.008977e+06,377897.064517,4.904889e+06,228096.792338,380958.263056,384718.619183,0.014197,4.631079e+06,4.676792e+06,856.288312
1,BANDUNG,107.610841,-7.099969,2021,1767.96,72.73,8008,14662620,8293.524740,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,4.369486e+06,329651.382642,4.278687e+06,198975.938240,436280.848042,440587.281368,0.012384,4.039835e+06,4.079711e+06,747.706688
2,BANDUNG,107.610841,-7.099969,2022,1767.96,73.16,16764,14830092,8388.250865,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,9.147111e+06,690094.377949,8.957032e+06,416537.541041,208407.039215,210464.179754,0.025925,8.457017e+06,8.540494e+06,1565.254750
3,BANDUNG,107.610841,-7.099969,2023,1767.96,73.74,4020,14997564,8482.976990,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,2.193473e+06,165484.335442,2.147892e+06,99885.523442,869406.983418,877988.710588,0.006217,2.027989e+06,2.048007e+06,375.209901
4,BANDUNG BARAT,107.414953,-6.897056,2020,1305.77,68.08,3864,7153344,5478.257273,"POLYGON ((107.40945 -6.68851, 107.40986 -6.688...",...,2.108354e+06,159062.555261,2.064541e+06,96009.368801,919960.166026,929040.892662,0.005976,1.949291e+06,1.968532e+06,354.591557


In [103]:
# Compute ICER correctly
df_final['icer_healthcare_le_cost_adjusted'] = abs(df_final['incremental_healthcare_cost'] / df_final['incremental_qaly'])
df_final['icer_payer_le_cost_adjusted'] = abs(df_final['incremental_payer_cost'] / df_final['incremental_qaly'])

df_final.head()

,District,x,y,Year,Area_sq_km,HDI,Cases,Population,Pop_den,geometry,...,payer_cost_vaccine_scaled,payer_cost_nonvaccine_scaled,ICER_healthcare_adjusted,ICER_payer_adjusted,cost_factor,incremental_healthcare_cost,incremental_payer_cost,incremental_qaly,icer_healthcare_le_cost_adjusted,icer_payer_le_cost_adjusted
0,BANDUNG,107.610841,-7.099969,2020,1767.96,72.39,9180,14495160,8198.805403,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,4.904889e+06,228096.792338,380958.263056,384718.619183,0.014197,4.631079e+06,4.676792e+06,856.288312,5408.317941,5461.702270
1,BANDUNG,107.610841,-7.099969,2021,1767.96,72.73,8008,14662620,8293.524740,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,4.278687e+06,198975.938240,436280.848042,440587.281368,0.012384,4.039835e+06,4.079711e+06,747.706688,5402.967419,5456.298935
2,BANDUNG,107.610841,-7.099969,2022,1767.96,73.16,16764,14830092,8388.250865,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,8.957032e+06,416537.541041,208407.039215,210464.179754,0.025925,8.457017e+06,8.540494e+06,1565.254750,5402.965214,5456.296708
3,BANDUNG,107.610841,-7.099969,2023,1767.96,73.74,4020,14997564,8482.976990,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,2.147892e+06,99885.523442,869406.983418,877988.710588,0.006217,2.027989e+06,2.048007e+06,375.209901,5404.945422,5458.296463
4,BANDUNG BARAT,107.414953,-6.897056,2020,1305.77,68.08,3864,7153344,5478.257273,"POLYGON ((107.40945 -6.68851, 107.40986 -6.688...",...,2.064541e+06,96009.368801,919960.166026,929040.892662,0.005976,1.949291e+06,1.968532e+06,354.591557,5497.285294,5551.547801
